# scraping

> Scraping from o2cm

In [1]:
#| default_exp scraping

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export

import requests
from bs4 import BeautifulSoup
from typing import List, Tuple, NamedTuple
from enum import Enum

from json import dumps
from math import log2
from dill import dump, load
from tqdm import tqdm
import os

In [4]:
#| export

class Style(Enum):
    LATIN = 'Latin'
    STANDARD = 'Standard'
    SMOOTH = 'Smooth'
    RHYTHM = 'Rhythm'

class Event(NamedTuple):
    division: str
    level: str
    dances: str
    style: Style | None = None

    def __str__(self):
        if self.style:
            return f"{self.division} {self.level} {self.style.value} {self.dances}".strip()
        else:
            return f"{self.division} {self.level} {self.dances}".strip()


In [24]:
#| export

METADATA_PICKLE = "../data/event_metadata.pkl"
event_metadata = {}
try:
    event_metadata = load(open(METADATA_PICKLE, "rb"))
except FileNotFoundError:
    pass

def make_augmentations(events: List[Event], urls: List[str] | None = None):
    import webbrowser

    if urls is None:
        urls = [""] * len(events)

    for event, url in tqdm(zip(events, urls)):
        if event in event_metadata:
            continue
        print(f"Event info: ({event.division}/{event.level}/{event.event}) {url}")
        webbrowser.open(url)

        correction = input("Correction (div, level, event): ")
        style_start = input("Style:")
        if style_start.strip() == "":
            raise ValueError("Invalid style")
        elif "rhythm".startswith(style_start.lower()):
            style = Style.RHYTHM
        elif "smooth".startswith(style_start.lower()):
            style = Style.SMOOTH
        elif "latin".startswith(style_start.lower()):
            style = Style.LATIN
        elif "standard".startswith(style_start.lower()):
            style = Style.STANDARD
        else:
            raise ValueError("Invalid style")

        if correction:
            correction = [x.strip() for x in correction.split(",")]
            if len(correction) == 0:
                raise ValueError("Invalid correction")
            fill = [(x if x else y) for x,y in zip(correction, [event.division, event.level, event.dances])]
            corrected_event = Event(*fill, style=style)
        else:
            corrected_event = Event(division=event.division, level=event.level, dances=event.dances, style=style)
        print(f"\t Correction: ({corrected_event.division}/{corrected_event.level}/{corrected_event.dances}/{corrected_event.style})")
        event_metadata[event] = corrected_event
        dump_event_metadata()

def dump_event_metadata(event_metadata=event_metadata):
    dump(event_metadata, open(METADATA_PICKLE, "wb"))

def load_event_metadata(event_metadata=event_metadata):
    event_metadata = load(open(METADATA_PICKLE, "rb"))
    return event_metadata

In [6]:
#| export

def parse_event_name(event_name):
    words = event_name.split()
    division = ' '.join(words[1:3])
    level = ' '.join(words[3:5]) if words[4].isdigit() else words[3]
    dances = ' '.join(words[5:]) if words[4].isdigit() else ' '.join(words[4:])
    base_event = Event(division=division, level=level, dances=dances)
    if base_event in event_metadata:
        return event_metadata[base_event]
    return base_event

event_names = ['9) Amateur Collegiate Gold Standard', '19) Amateur Collegiate Silver Rhythm', '4) Amateur Collegiate Gold Rhythm', '10) Amateur Adult Silver Rhythm', '2) Amateur Collegiate Silver Standard', '3) Amateur Collegiate Silver Intl. Tango', '4) Amateur Collegiate All Syllabus Standard', '10) Amateur Adult Silver Intl. V. Waltz', '3) Amateur Adult Silver Standard', '3) Amateur Adult Silver Intl. Tango', '11) Amateur Collegiate Silver Smooth', '2) Amateur Adult Gold Smooth', '1) Amateur Collegiate Gold Smooth', '6) Amateur Adult Novice Smooth', '23) Amateur Adult Silver Latin', '9) Amateur Collegiate Silver Latin', '8) Amateur Collegiate Bronze Latin', '15) Amateur Adult Bronze 1 Latin', '8) Amateur Adult Bronze Latin']
for name in event_names:
    print(parse_event_name(name))

Amateur Collegiate Gold Standard WTFQ
Amateur Collegiate Silver Rhythm CRS
Amateur Collegiate Gold Rhythm CRSB
Amateur Adult Silver Rhythm CRS
Amateur Collegiate Silver Standard WFQ
Amateur Collegiate Silver Standard T
Amateur Collegiate Gold Standard V
Amateur Adult Silver Standard V
Amateur Adult Silver Standard WFQ
Amateur Adult Silver Latin Intl. Tango
Amateur Collegiate Silver Smooth WTF
Amateur Adult Gold Smooth WTFV
Amateur Collegiate Gold Latin Smooth
Amateur Adult Novice Smooth WTF
Amateur Adult Silver Latin SCR
Amateur Collegiate Silver Latin SCR
Amateur Collegiate Bronze Latin CR
Amateur Adult Bronze Latin SJ
Amateur Adult Bronze Latin CR


In [7]:
#| export

o2cm_cache = {}
CACHE_PICKLE = "../data/o2cm_cache.pkl"
if os.path.exists(CACHE_PICKLE):
    try:
        o2cm_cache = load(open(CACHE_PICKLE, "rb"))
    except (EOFError, ImportError):
        pass


In [8]:
#| export

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def get(url, **kwargs):
    return requests.get(url, headers=headers, **kwargs)

def get_event_list_html(name: str) -> BeautifulSoup:
    first, last = name.split(" ", 1)
    url = f"https://results.o2cm.com/individual.asp?szLast={last}&szFirst={first}"
    if url in o2cm_cache:
        return o2cm_cache[url]
    page: requests.Response = get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    o2cm_cache[url] = soup
    if log2(len(o2cm_cache)).is_integer():
        dump(o2cm_cache, open(CACHE_PICKLE, "wb"))
    return soup

name = "Sasha Hydrie"
soup = get_event_list_html(name)

In [9]:
#| export

def extract_events_from_html(soup: BeautifulSoup) -> List[Tuple[str, str]]:
    events = []
    include_events = False

    for tag in soup.find_all(['b', 'a']):  
        if tag.name == 'b' and tag.text == '03-22-24 - USA DANCE National DanceSport Championships':
            include_events = True
        elif tag.name == 'b' and include_events:
            break 
        elif tag.name == 'a' and include_events:
            events.append((parse_event_name(tag.text), tag.get('href')))

    return events

events = extract_events_from_html(soup)
events

[(Event(division='Amateur Adult', level='Silver', dances='Intl. Tango', style=<Style.LATIN: 'Latin'>),
  'http://Results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40423019'),
 (Event(division='Amateur Adult', level='Bronze', dances='SJ', style=<Style.LATIN: 'Latin'>),
  'http://Results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40422B20'),
 (Event(division='Amateur Collegiate', level='Silver', dances='WTF', style=<Style.SMOOTH: 'Smooth'>),
  'http://Results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40323030'),
 (Event(division='Amateur Collegiate', level='Silver', dances='CRS', style=<Style.RHYTHM: 'Rhythm'>),
  'http://Results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40323040'),
 (Event(division='Amateur Collegiate', level='Silver', dances='B', style=<Style.RHYTHM: 'Rhythm'>),
  'http://Results.o2cm.com/scoresheet3.asp?event=usa24&heatid=4032304B'),
 (Event(division='Amateur Adult', level='Silver', dances='CRS', style=<Style.RHYTHM: 'Rhythm'>),
  'http://Results.o2cm.com/score

In [10]:
#| export 

class Result(NamedTuple):
    callbacks: int
    placement: float | None
    num_dances: int = 1

In [11]:
#| export

def extract_max_callbacks(soup: BeautifulSoup) -> int:
    select_element = soup.find('select', {'id': 'selCount'})
    possible_callbacks = len(select_element.find_all('option')) - 1 if select_element else 0 # type: ignore
    return possible_callbacks

def extract_placement(soup: BeautifulSoup, name: str, verbose=False) -> float | None: 
    """assumes that soup is a finals page"""
    couple_number = None
    for link in soup.find_all('a'):
        if link.text == name:
            parent_td = link.find_parent('td')  # Parent <td> which should have sibling with couple number
            if parent_td:
                # The immediate previous sibling <td> of `parent_td` contains the couple number
                prev_td = parent_td.find_previous_sibling("td", class_="t1b")
                if prev_td:
                    couple_number = prev_td.text.strip()
                    if verbose:
                        print(f"Found {couple_number} associated with {name}")
                    break
    else:
        if verbose:
            print(f"Dancer {name} didn't final")
        return None

    summary_table = soup.find('td', string='Summary').find_parent('table') if soup.find('td', string='Summary') else None # type: ignore
    if summary_table:
        for row in summary_table.find_all('tr'):
            cells = row.find_all('td') 
            if cells and cells[0].get_text(strip=True) == couple_number:
                averaged_place = cells[-1].get_text(strip=True)  
                return float(averaged_place)

    results_table = soup.find('table', class_='t1n')
    for row in results_table.find_all('tr'): # type: ignore
        cells = row.find_all('td') 
        if cells and cells[0].get_text(strip=True) == couple_number:
            averaged_place = cells[-2].get_text(strip=True)  
            return float(averaged_place)

def extract_num_dances(soup: BeautifulSoup) -> int:
    return max(len(soup.find_all('table', class_='t1n')) - 2, 1)


In [12]:

url = "https://results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40423019"
name = "Khalid Ali"

initial_res = get(url)
soup = BeautifulSoup(initial_res.content, "html.parser")

extract_placement(soup, name)


3.5

In [13]:
#| export

def get_event_result(name: str, url: str, o2cm_cache=o2cm_cache) -> Result:
    query_string = url.split("?")[1]
    key_value_pairs = [query.split("=") for query in query_string.split("&")]
    data: dict[str, str | int] = {key: value for key, value in key_value_pairs}

    # CR shy: factor session creation out
    session = requests.Session()
    session.headers.update(headers)

    if url in o2cm_cache:
        soup = o2cm_cache[url]
    else:
        initial_res = session.get(url)
        soup = BeautifulSoup(initial_res.content, "html.parser")
        o2cm_cache[url] = soup
        if log2(len(o2cm_cache)).is_integer():
            dump(o2cm_cache, open(CACHE_PICKLE, "wb"))

    possible_callbacks = extract_max_callbacks(soup)
    num_dances = extract_num_dances(soup)

    if (place := extract_placement(soup, name)) is not None:
        return Result(callbacks=possible_callbacks, placement=place, num_dances=num_dances)


    base_url = "https://results.o2cm.com/scoresheet3.asp"
    # CR shy: binary search eventually
    for selector in range(1, possible_callbacks + 1):
        data["selCount"] = selector
        if dumps(data) in o2cm_cache:
            soup = o2cm_cache[dumps(data)]
        else:
            response = session.post(base_url, data=data)
            # Process the response
            soup = BeautifulSoup(response.content, 'html.parser')
            o2cm_cache[dumps(data)] = soup
            if log2(len(o2cm_cache)).is_integer():
                dump(o2cm_cache, open(CACHE_PICKLE, "wb"))

        if name in soup.get_text():
            return Result(callbacks=possible_callbacks - selector, placement=None, num_dances=num_dances)
    else:
        raise ValueError(f"Couldn't find {name} in {url}")

get_event_result("Khalid Ali", "https://results.o2cm.com/scoresheet3.asp?event=usa24&heatid=40423019")
    


Result(callbacks=2, placement=3.5, num_dances=1)

In [14]:
#| export

def dump_cache(o2cm_cache=o2cm_cache):
    dump(o2cm_cache, open(CACHE_PICKLE, "wb"))